In [0]:
import zipfile
import os
from pyspark.sql.functions import *

In [0]:
display(dbutils.fs.ls("FileStore/tables"))

path,name,size,modificationTime
dbfs:/FileStore/tables/bronze/,bronze/,0,0
dbfs:/FileStore/tables/discogs.csv/,discogs.csv/,0,0
dbfs:/FileStore/tables/discogs_csv.zip,discogs_csv.zip,1762752803,1743384856000
dbfs:/FileStore/tables/discogs_parquet/,discogs_parquet/,0,0
dbfs:/FileStore/tables/gold/,gold/,0,0
dbfs:/FileStore/tables/silver/,silver/,0,0


In [0]:
zip_dbfs= "dbfs:/FileStore/tables/discogs_csv.zip"
zip_local = "/tmp/discogs_csv.zip"
extract = "/tmp/discogs_csv_novo/"

dbutils.fs.cp(zip_dbfs, f"file:{zip_local}")
os.makedirs(extract, exist_ok=True)

with zipfile.ZipFile(zip_local, 'r') as zip_ref:
    zip_ref.extractall(extract)

print(f"arquivos extraídos em: {extract}")


Arquivos extraídos em: /tmp/discogs_csv_novo/


In [0]:
csv= "file:/tmp/discogs_csv_novo/"
csv_dbfs= "dbfs:/FileStore/tables/discogs.csv"

dbutils.fs.cp(csv, csv_dbfs, recurse=True)

Out[14]: True

In [0]:
df = spark.read.csv("dbfs:/FileStore/tables/discogs.csv/",header = True,inferSchema = True,sep = ",",quote='"',escape='"',multiLine=True)

In [0]:
parquet_path = "dbfs:/FileStore/tables/discogs_parquet"
df.write.mode("overwrite").parquet(parquet_path)

In [0]:
df_parquet = spark.read.parquet("dbfs:/FileStore/tables/discogs_parquet/")
display(df_parquet.head(10))


id,release_id,status,title,artist_id,artist_name,label_name,label_id,format,genre,style,country,release_date,notes,master_id,video_url,company_name
1,12295801,Accepted,The World Of Ray Price,311678,Ray Price,Columbia,1866,8-Track Cartridge,"Folk, World, & Country",null,US,1970,null,null,null,null
2,12295802,Accepted,The Burden of Isolation,3720243,Filth (9),Not on Label (Filth (9) Self-Released),1495843,CD,Rock,Deathcore,US,2018,null,1397110,https://www.youtube.com/watch?v=mRN-e_mIfQQ,null
3,12295803,Accepted,Bassoon Concertos,6095671,Sebastian Fagerlund,BIS,51038,SACD,Classical,Contemporary,Sweden,2016,Made in the EU,null,null,BIS Records AB
4,12295805,Accepted,Ich Lag In Einer Nacht Und Schlief,3170804,Hans Peter Treichler,Gold Records,11489,Vinyl,Pop,null,Switzerland,1980,null,null,null,Gold Records
5,12295806,Accepted,Bien O Mal,85181,Julieta Venegas,Sony Music,25487,CD,Latin,null,Argentina,2010,null,1418114,https://www.youtube.com/watch?v=KTr9HMnAWNE,Lolein Music
6,12295807,Accepted,Kaleidoscopia,3130927,Beatnik (8),Eargasm Wreckords,500315,CD,Rock,Krautrock,Switzerland,2008,Mastered by Eroc Produced by Beatnik 16-page booklet whit lyrics and credits P&C 2008 Eargasm Wreckords,null,https://www.youtube.com/watch?v=W4ZpVHFI_zM,Eargasm Wreckords
7,12295808,Accepted,Κόκκινη Γραμμή,2444813,Βασίλης Τερλέγκας,Alpha Records (6),153245,CD,"Folk, World, & Country",null,Greece,null,null,null,https://www.youtube.com/watch?v=6Z3q5YEpnno,null
8,12295809,Accepted,Frescobaldi Edition Vol 6 - Il Primo Libro Dei Madrigali a cinque voci,834330,Girolamo Frescobaldi,Brilliant Classics,89052,CD,Classical,Renaissance,Netherlands,2009,"Volume 6 of this path-breaking series, the first to record every extant work by Girolamo Frescobaldi, focuses on the secular madrigals. These works - he wrote one book, and evidently hoped to write more - are no less innovatory than the keyboard works that so impressed and influenced J.S. Bach. The texts feature the usual parade of lovelord shepherds and sighing maidens. Love unrequited is the keynote, but Frescobaldi's musical response to the expressive affekt is more charming than heart-rending: these are not the passionate and personal declarations of Monteverdi or Gesualdo. These madrigals were Frescobaldi's first published music, and therefore evidently music of which he felt justifiably proud. Their language is doubtless shaped by his education and service in the northern Italian cities of Ferrara and Mantua, which thanks to the patronage of families such as the famous Gonzagas was a cosmopolitan hotbed of artistic and creative endeavour. Frescobaldi met and became friends with Peter Philips, the noted English composer of madrigals who had settled in the Netherlands, and Philips's delicacy of imagination can be heard in this collection. The nineteen madrigals with which Frescobaldi introduced himself have an admirable clarity of formal design that gives each line its due weight in terms of duration and emotion, and a transparent counterpoint that favours delicacy over density as a stylistic means, homophonic and polyphonic sections being cleverly alternated. It is pleasing to see the respect that Frescobaldi pays to the texts: the words are set to graceful melodic phrases, and never obscured by excessive counterpoint, but interpreted literally with immediate attention to meaning.",null,null,null
9,12295810,Accepted,Try A Little Kindness / One Step At A Time,3432907,Don Lauren,Apt Records,125142,Vinyl,Rock,Vocal,US,null,A side: pop vocal B side: bubblegum rock,1237274,null,null
10,12295811,Accepted,A Tribute to La Monte Young,6522519,Infinite Music,Fire Records,2051,CD,Electronic,Minimal,UK,2018,,null,https://www.youtube.com/watch?v=lzSC5wCCOVo,null


In [0]:
for c in df_parquet.columns:
    null = df_parquet.select(count(when(col(c).isNull(), c)).alias(c))
    display(null)

id
0


release_id
0


status
0


title
0


artist_id
0


...

release_date
2173336


notes
6964173


master_id
14118736


video_url
8613166


company_name
8666021


In [0]:
df_null = df_parquet.dropna(how ="all")
df_drop = df_null.dropDuplicates()


In [0]:
for c in df_drop.columns:
    if dict(df_drop.dtypes)[c] in ['int', 'bigint', 'double', 'float', 'decimal']:
        df_drop = df_drop.fillna({c: 0})
    else:
        df_drop = df_drop.fillna({c: "desconhecido"})

In [0]:
df_string = df_drop.withColumn("release_date", col("release_date").cast("string"))

In [0]:

df_date = df_string.withColumn("release_date",when(col("release_date") == "0", lit("1900-01-01")).otherwise(concat(col("release_date").cast("string"), lit("-01-01")))).withColumn("release_date", to_date(col("release_date"), "yyyy-MM-dd"))


In [0]:
for c in df_date.columns:
    df_null = df_date.select(count(when(col(c).isNull(), c)).alias(c))
    display(df_null)

id
0


release_id
0


status
0


title
0


artist_id
0


...

release_date
0


notes
0


master_id
0


video_url
0


company_name
0


In [0]:
'''
Se quiser prever release_date (ano que o disco foi lançado)

obs: aplicar engenharia de atributos 

features: genre,format,label_id
target: release_date

'''

In [0]:
df_bronze = df_parquet
df_bronze.write.mode("overwrite").parquet("dbfs:/FileStore/tables/bronze/discogs_bronze.parquet")

In [0]:
df_silver = df_string
df_silver.write.mode("overwrite").parquet("dbfs:/FileStore/tables/silver/discogs_silver.parquet")

In [0]:
df_gold = df_date
df_gold.write.mode("overwrite").parquet("dbfs:/FileStore/tables/gold/discogs_gold.parquet")


In [0]:
df_parquet_gold = spark.read.parquet("dbfs:/FileStore/tables/gold/discogs_gold.parquet")
display(df_parquet_gold.head(10))


id,release_id,status,title,artist_id,artist_name,label_name,label_id,format,genre,style,country,release_date,notes,master_id,video_url,company_name
682,103536,Accepted,Fire Night Dance,98684,Peter Jacques Band,Prelude Records,4081,Vinyl,Electronic,Disco,US,1979-01-01,"Special thanks to William Wright - First string Studio credit: Recorded at Fonoprint Special thanks to Mr. Maurizio who always gives us the best Sigma Sound Studio: Mixed at Sigma Sound Studios New York-USA Mastered at Sterling sound New York-USA Other credits: We would like to dedicate this album to all the people of Goody Music Productions for their enthusiasm and work. Special credit to all the musicians of the Macho album. Executive produced for Goody Music Production All songs published by Little Macho Music ℗ 1979 Prelude Records © 1979 Prelude Records A product of Goody Music/Prelude Records, Inc. Manufactured and distributed by Prelude Records Front cover misspelled band name by ""Peter Jaeques Band"" instead of ""Peter Jacques Band"" as on rear and labels. This LP was first released as a double 12"" with a gatefold jacket on Prelude Records with the same cat#.",0,https://www.youtube.com/watch?v=MrCmEjPM2Gw,Fonoprint
929,12271983,Accepted,Runaway,95516,Jasmine (2),Canyon International,67638,Vinyl,Electronic,Eurobeat,Japan,1989-01-01,desconhecido,0,https://www.youtube.com/watch?v=yKSTDqGrSus,Pony Canyon Inc.
986,103574,Accepted,It Doesn't Really Matter,45227,Zapp,Warner Bros. Records,1000,Vinyl,Electronic,Electro,Germany,1985-01-01,℗ 1985 Warner Bros. Records Inc. for the U.S. ℗ 1985 WEA International Inc. © 1985 WEA Records Ltd.,14311,https://www.youtube.com/watch?v=hX9bIN2qT8o,Warner Bros. Records Inc.
1006,12284137,Accepted,Inevitable Archives: Vol. 4 - The Joy Of Disease: Demos And Remixes,12544,James Plotkin,Not On Label (James Plotkin Self-released),1494539,File,Electronic,Experimental,desconhecido,2017-01-01,"""The Joy Of Disease: Demos And Remixes"" showcases the initial recordings in preparation for [r=97742] album, released in 1995 on John Zorn's Japanese Avant label. These early home-recorded instrumental versions are supplemented by unreleased remixes from Plotkin and co-producer Mick Harris of Napalm Death/Scorn. The remixes were done during the original album sessions at Mick Harris' Black Box studio in the Custard Factory in Birmingham, UK. Written and recorded to 8-track tape 1994-1995. Mastered for vinyl September 2017. All music copyright 1996",1729746,https://www.youtube.com/watch?v=Eaez1wGLGm0,desconhecido
1015,12307814,Accepted,"Johnson Rag / Yes, My Darling Daughter",335671,Glenn Miller And His Orchestra,His Master's Voice,37150,Shellac,Jazz,Swing,Netherlands,1900-01-01,desconhecido,0,https://www.youtube.com/watch?v=88KnilAi7w4,The Gramophone Co. Ltd.
1490,103637,Accepted,Acid Blues / Transformer,18230,Transparent,Dragonfly Records,356,Vinyl,Electronic,Psy-Trance,UK,2002-01-01,Mastered @ Heatmans Mastering. Published by B-Unique Music. (P)2002 Dragonfly Records. (C)2002 Dragonfly Records. Barcode: 5 025055 200934,0,https://www.youtube.com/watch?v=PRxdg4Vo8rM,desconhecido
1795,10500,Accepted,Comatose,10656,Front Line Assembly,Metropolis,1490,CD,Electronic,Industrial,US,1998-01-01,℗ Metropolis Records 1998 © Edition Subterranean/Warner Chappell 1998 Under exclusive license from OFFBEAT.,0,https://www.youtube.com/watch?v=8wKJAw3eaVk,Off Beat
1863,12307934,Accepted,فضا,3673301,امير آرام,Pars Video,960862,CD,"Folk, World, & Country",desconhecido,US,1994-01-01,Produced & Distributed by Pars Video (C)(R) Pars Vide U.S.A.,0,desconhecido,Pars Video
5553,12272730,Accepted,Hayalci,6594863,Deli Gömleği,Deli Gömleği,1493200,File,Rock,Grunge,Turkey,2018-01-01,desconhecido,0,desconhecido,Özver Yılmaz
2061,12296406,Accepted,Moonlight Sinatra,52833,Frank Sinatra,Reprise Records,157,Vinyl,Jazz,Big Band,US,1966-01-01,"Pink/blue/green Reprise ""Smiling Frank"" labels. First cat. # on sleeve spine and back Second cat. # on labels Third cat. # on sleeve fr

In [0]:
display(df_parquet_gold.select("status").distinct())

status
Draft
Accepted
Deleted


In [0]:
'''

supondo que o time de analistas precise desses dados filtrados em disco para otimizar futuras consultas acessar apenas o status "Accepted"

'''

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet")

Out[88]: True

In [0]:
df_parquet_gold.write.mode("overwrite").partitionBy("status").parquet("dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet")

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet/")) 

path,name,size,modificationTime
dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet/_SUCCESS,_SUCCESS,0,1743397562000
dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet/_committed_804095705787285339,_committed_804095705787285339,35,1743397561000
dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet/status=Accepted/,status=Accepted/,0,0
dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet/status=Deleted/,status=Deleted/,0,0
dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet/status=Draft/,status=Draft/,0,0


In [0]:
df_accepted = spark.read.parquet("dbfs:/FileStore/tables/gold/discogs_gold_divided.parquet/status=Accepted")
display(df_accepted.head(10))


id,release_id,title,artist_id,artist_name,label_name,label_id,format,genre,style,country,release_date,notes,master_id,video_url,company_name
682,103536,Fire Night Dance,98684,Peter Jacques Band,Prelude Records,4081,Vinyl,Electronic,Disco,US,1979-01-01,"Special thanks to William Wright - First string Studio credit: Recorded at Fonoprint Special thanks to Mr. Maurizio who always gives us the best Sigma Sound Studio: Mixed at Sigma Sound Studios New York-USA Mastered at Sterling sound New York-USA Other credits: We would like to dedicate this album to all the people of Goody Music Productions for their enthusiasm and work. Special credit to all the musicians of the Macho album. Executive produced for Goody Music Production All songs published by Little Macho Music ℗ 1979 Prelude Records © 1979 Prelude Records A product of Goody Music/Prelude Records, Inc. Manufactured and distributed by Prelude Records Front cover misspelled band name by ""Peter Jaeques Band"" instead of ""Peter Jacques Band"" as on rear and labels. This LP was first released as a double 12"" with a gatefold jacket on Prelude Records with the same cat#.",0,https://www.youtube.com/watch?v=MrCmEjPM2Gw,Fonoprint
929,12271983,Runaway,95516,Jasmine (2),Canyon International,67638,Vinyl,Electronic,Eurobeat,Japan,1989-01-01,desconhecido,0,https://www.youtube.com/watch?v=yKSTDqGrSus,Pony Canyon Inc.
986,103574,It Doesn't Really Matter,45227,Zapp,Warner Bros. Records,1000,Vinyl,Electronic,Electro,Germany,1985-01-01,℗ 1985 Warner Bros. Records Inc. for the U.S. ℗ 1985 WEA International Inc. © 1985 WEA Records Ltd.,14311,https://www.youtube.com/watch?v=hX9bIN2qT8o,Warner Bros. Records Inc.
1006,12284137,Inevitable Archives: Vol. 4 - The Joy Of Disease: Demos And Remixes,12544,James Plotkin,Not On Label (James Plotkin Self-released),1494539,File,Electronic,Experimental,desconhecido,2017-01-01,"""The Joy Of Disease: Demos And Remixes"" showcases the initial recordings in preparation for [r=97742] album, released in 1995 on John Zorn's Japanese Avant label. These early home-recorded instrumental versions are supplemented by unreleased remixes from Plotkin and co-producer Mick Harris of Napalm Death/Scorn. The remixes were done during the original album sessions at Mick Harris' Black Box studio in the Custard Factory in Birmingham, UK. Written and recorded to 8-track tape 1994-1995. Mastered for vinyl September 2017. All music copyright 1996",1729746,https://www.youtube.com/watch?v=Eaez1wGLGm0,desconhecido
1015,12307814,"Johnson Rag / Yes, My Darling Daughter",335671,Glenn Miller And His Orchestra,His Master's Voice,37150,Shellac,Jazz,Swing,Netherlands,1900-01-01,desconhecido,0,https://www.youtube.com/watch?v=88KnilAi7w4,The Gramophone Co. Ltd.
1490,103637,Acid Blues / Transformer,18230,Transparent,Dragonfly Records,356,Vinyl,Electronic,Psy-Trance,UK,2002-01-01,Mastered @ Heatmans Mastering. Published by B-Unique Music. (P)2002 Dragonfly Records. (C)2002 Dragonfly Records. Barcode: 5 025055 200934,0,https://www.youtube.com/watch?v=PRxdg4Vo8rM,desconhecido
1795,10500,Comatose,10656,Front Line Assembly,Metropolis,1490,CD,Electronic,Industrial,US,1998-01-01,℗ Metropolis Records 1998 © Edition Subterranean/Warner Chappell 1998 Under exclusive license from OFFBEAT.,0,https://www.youtube.com/watch?v=8wKJAw3eaVk,Off Beat
1863,12307934,فضا,3673301,امير آرام,Pars Video,960862,CD,"Folk, World, & Country",desconhecido,US,1994-01-01,Produced & Distributed by Pars Video (C)(R) Pars Vide U.S.A.,0,desconhecido,Pars Video
5553,12272730,Hayalci,6594863,Deli Gömleği,Deli Gömleği,1493200,File,Rock,Grunge,Turkey,2018-01-01,desconhecido,0,desconhecido,Özver Yılmaz
2061,12296406,Moonlight Sinatra,52833,Frank Sinatra,Reprise Records,157,Vinyl,Jazz,Big Band,US,1966-01-01,"Pink/blue/green Reprise ""Smiling Frank"" labels. First cat. # on sleeve spine and back Second cat. # on labels Third cat. # on sleeve front Pressing plant variant to [r=7934232] and [r=3399572]. ""T"" stamped in runout denotes [l4039

In [0]:
'''

separar em tabelas fato e dimensao

'''

In [0]:
dimensao_artista = df_accepted.select("artist_id", "artist_name").distinct()
dimensao_artista.write.format("delta").mode("overwrite").saveAsTable("dimensao_artista")
display(dimensao_artista.head(10))

artist_id,artist_name
18230,Transparent
37737,Dave Brubeck
1062078,Orkiestra Mieczysława Paszkieta
6277,Dido
4160582,上原正吉
2777432,International Staff Band Of The Salvation Army
1974337,Grand Dansorkester
4982,Alec Empire
392601,Art Laboe
4545986,Malcolm Kennedy


In [0]:
dimensao_label = df_accepted.select("label_id", "label_name", "company_name").distinct()
dimensao_label.write.format("delta").mode("overwrite").saveAsTable("dimensao_label")
display(dimensao_label.head(10))


label_id,label_name,company_name
187471,AT-Production,desconhecido
647134,Gusto-Starday,"Victor Musical Industries, Inc."
106394,Word,BMG Direct
46543,Edigsa,Hansa Tonstudios
1428258,bearfaced music,desconhecido
26949,Showtime,desconhecido
29583,RCA Red Seal,Sony BMG Music Entertainment
147207,Enamell Records,desconhecido
4898,United Artists Records,"United Artists Records, inc."
1562507,Booom Boxx Records,desconhecido


In [0]:
discogs_fato = df_accepted.select("id","release_id","title","artist_id","label_id","format","genre","style","country","release_date","notes","master_id","video_url","company_name")

discogs_fato.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("discogs_fato")